## Linear regression

Scope:
- I'm going to train a simple linear regressor as a baseline model on the scikit-learn diabetes dataset. The data was weirdly normalised, so I just loaded in the raw .csv from the source.
- I'm also going to do some visualisation of the dataset with Altair and Seaborn, side-by-side;

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
diabetes_df = pd.read_csv('../data/diabetes.csv')

# I want the column names to be a bit more descriptive
diabetes_df.rename(columns={'S1':'t_cells', 'S2':'ld_lipo', 'S3':'hd_lipo',
                            'S4':'thyroid-sh', 'S5':'lamotrigine', 'S6':'blood_sugar'}, inplace=True)

Let's see what this dataset looks like.

In [ ]:
diabetes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          442 non-null    int64  
 1   SEX          442 non-null    int64  
 2   BMI          442 non-null    float64
 3   BP           442 non-null    float64
 4   t_cells      442 non-null    int64  
 5   ld_lipo      442 non-null    float64
 6   hd_lipo      442 non-null    float64
 7   thyroid-sh   442 non-null    float64
 8   lamotrigine  442 non-null    float64
 9   blood_sugar  442 non-null    int64  
 10  Y            442 non-null    int64  
dtypes: float64(6), int64(5)
memory usage: 38.1 KB


We have no null values, which is great. We have 10 features or predictive variables and one target variable, Y. Y is a quantitative measure of disease progression one year after baseline. But what does Y actually look like?

In [ ]:
diabetes_df.describe()['Y']

count    442.000000
mean     152.133484
std       77.093005
min       25.000000
25%       87.000000
50%      140.500000
75%      211.500000
max      346.000000
Name: Y, dtype: float64

In [ ]:
# let's eliminate the predicted column, then split the data
X = diabetes_df.drop('Y', axis=1)
y = diabetes_df['Y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=True)

# training the linear regressor on the training data
model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)

# predicting the test data
predictions_lr = model_lr.predict(X_test)

42.876129971235926


In [ ]:
# evaluating model
print('MAE:', mean_absolute_error(y_test, predictions_lr))
print('R2:', round(r2_score(y_test, predictions_lr),2))

MAE: 42.876129971235926
R2: 0.57


In [ ]:
# just playing around with different visualisations
# what would be useful to visualise?
def plot_altair(column):
    return alt.Chart(diabetes_df).mark_point(filled=True).encode(
        x = alt.X(column, scale=alt.Scale(zero=False)),
        y = alt.Y('Y:Q', scale=alt.Scale(zero=False)))
        # color = alt.Color('SEX:N'),
        # size = alt.Size('blood_sugar:Q', title='Blood sugar'),
        # opacity = alt.OpacityValue(0.5))

# a regression line for each variable against the target variable
# but this is the /actual/ target variable, not the model's prediction of it
charts = []
for col in list(X.columns):
    chart = plot_altair(col + ':Q')
    charts.append(chart + chart.transform_regression(str(col), 'Y').mark_line())

alt.vconcat(*charts[2:])

alt.VConcatChart(...)